In [1]:
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [16]:
# load cleaned data
df = pd.read_csv('cleaned.csv')
df = df.drop('Unnamed: 0', axis=1)

In [18]:
df = df.sort_values(by=['srch_id'])

In [19]:
def relevance_score_click_bool(qd):
    id = qd
    if qd['booking_bool'] == 1:
        return 5
    if qd['click_bool'] == 1:
        return 1
    return 0

df["relevance"] = df.apply(relevance_score_click_bool,axis=1)

In [20]:
ids = df['srch_id'].unique()
ids.sort()

In [21]:
ids = df['srch_id'].unique()
ids.sort()
train_idx, validation_idx = train_test_split(ids, test_size=0.2)

In [22]:
train = df[df['srch_id'].isin(train_idx)]
val = df[df['srch_id'].isin(validation_idx)]

In [23]:
train_rank = train['relevance']
train_qid = train['srch_id']
train_x = train.drop(['click_bool', 'booking_bool', 'relevance','srch_id'],axis=1)

val_rank = val['relevance']
val_qid = val['srch_id']
val_x = val.drop(['click_bool', 'booking_bool', 'relevance','srch_id'],axis=1)

In [24]:
ranker = xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:ndcg", lambdarank_pair_method="topk")
ranker.fit(train_x, train_rank, train_qid)

/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/xgboost/core.py:727: FutureWarning: Pass `group` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBoostError: [18:33:21] /Users/runner/work/xgboost/xgboost/src/data/validation.h:36: Check failed: valid_query_group: Invalid group structure.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001198da355 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x00000001199b1a36 xgboost::data::ValidateQueryGroup(std::__1::vector<unsigned int, std::__1::allocator<unsigned int> > const&) + 182
  [bt] (2) 3   libxgboost.dylib                    0x00000001199b0218 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 824
  [bt] (3) 4   libxgboost.dylib                    0x00000001199afdf2 xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (4) 5   libxgboost.dylib                    0x00000001198f2729 XGDMatrixSetInfoFromInterface + 233
  [bt] (5) 6   libffi.7.dylib                      0x000000010c093ead ffi_call_unix64 + 85
 